In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
if '__file__' not in globals(): __file__, __name__ = globals()['__vsc_ipynb_file__'], '__ipynb__'

import sys, os
if os.path.abspath('.') not in sys.path: sys.path.append(os.path.abspath('.'))


### with sys.profile

In [31]:
import sys
import threading
import time


def _get_frameinfo(frame):
    if frame is None: return ('?', '?', -1)

    func_name = frame.f_code.co_name
    if 'self' in frame.f_locals:
        class_name = frame.f_locals['self'].__class__.__name__
        func_name = class_name + '.' + func_name

    src_path = frame.f_code.co_filename
    # src_path = os.path.basename(frame.f_code.co_filename)
    return (func_name, src_path, frame.f_lineno)


ROI_DIRS = [
    # '/home/jjkim/anaconda3/envs/stock1/lib/python3.10/asyncio/',
    # '/tmp/ipykernel',
]

indent = ''
def cb_trace(frame, event, arg):
    global indent
    
    # ROI filter
    if len(ROI_DIRS)>0 and  not any( roi in frame.f_code.co_filename for roi in ROI_DIRS):
        return 
    # if "/tmp/ipykernel" not in frame.f_code.co_filename: 
    #     return # return if not notebook src.
    
    func, srcpath, line = _get_frameinfo(frame)
    if event == 'call': 
        print(indent, f'{{ {func} {srcpath}:{line} th={threading.current_thread().name}')
        indent += '\t'
    # elif event == 'c_call': 
    #     print(indent, f'>> {func} {srcpath}:{line} th={threading.current_thread().name}')
    #     indent += '\t'
    # elif event == 'c_return': 
    #     print(indent, f'<< {func} {srcpath}:{line} th={threading.current_thread().name}')
    #     indent += '\t'
    elif event == 'return':
        indent = indent[:-1]
        print(indent, f'}} // {func} {srcpath}:{line} th={threading.current_thread().name}')

#
#
#
class Upper:
    def __init__(self, b) -> None:
        print(indent, 'b', b, ' in Base')

class Middle:
    def __init__(self, b) -> None:
        print(indent, 'b', b, ' in Base')

class Lower(Middle):
    def __init__(self, a=1, b=0) -> None:
        super().__init__(b)

    def memberfunc(self, a,b):
        print(indent, 'memberfunc', a,b)


def scope_test():

    print(indent, '-------------- max_count=2 test ---------------')
    def max_count_test(a,b,c):
        print(indent, f'It will shown only max_count(2) times', a,b,c)

        obj = Lower(100,101)
        obj.memberfunc(4,5)
        return a+b+c

    for _ in range(5):
        max_count_test(1,2,3)

    print(indent, '-------------- suppress test ---------------')

    def in_suppress_1():
        print(indent, 'Not seen: in_suppress_1')

    def suppress_following():
        print(indent, 'enter: suppress_following()')
        in_suppress_1()
        print(indent, 'exit: suppress_following: after in_suppress_1')
        return

    suppress_following()
    return


def print_numbers():
    sys.setprofile(cb_trace)

    for i in range(1, 3, 3):
        print(indent, f"number: {i}")
        scope_test()
        time.sleep(1)
    sys.setprofile(None)

def print_letters():
    sys.setprofile(cb_trace)

    for letter in 'abcde':
        print(indent, f"Letter: {letter}")
        scope_test()
        time.sleep(1)
    sys.setprofile(None)


import asyncio
import nest_asyncio
nest_asyncio.apply() # Allow running asyncio in Jupyter Notebook

async def find_users_async(n):
    for i in range(1, n + 1):
        print(f'do something {i}/{n}...')
        await asyncio.sleep(1)
    print(f'= do something {n} done ...')

async def process_async():
    await asyncio.gather(
        find_users_async(3),
        find_users_async(2),
        find_users_async(1),
    )


if __name__ == "__ipynb__":
    sys.setprofile(cb_trace)

    print_letters()

    # threads = []
    # for i, fn in enumerate([print_numbers, print_letters]):
    #     t = threading.Thread(target=fn, name=f't{i}')
    #     threads.append(t)
    #     t.start()
    #     break # 

    # for t in threads: t.join()

    # await process_async()
    # print('after asyncio.run...')
    
    sys.setprofile(None)



 { print_letters /tmp/ipykernel_60057/4044955964.py:105 th=MainThread
	 { OutStream.write /home/jjkim/.local/lib/python3.10/site-packages/ipykernel/iostream.py:535 th=MainThread
		 { OutStream._is_master_process /home/jjkim/.local/lib/python3.10/site-packages/ipykernel/iostream.py:444 th=MainThread
		 } // OutStream._is_master_process /home/jjkim/.local/lib/python3.10/site-packages/ipykernel/iostream.py:445 th=MainThread
			 { OutStream._schedule_flush /home/jjkim/.local/lib/python3.10/site-packages/ipykernel/iostream.py:465 th=MainThread
		 } // OutStream._schedule_flush /home/jjkim/.local/lib/python3.10/site-packages/ipykernel/iostream.py:471 th=MainThread
	 } // OutStream.write /home/jjkim/.local/lib/python3.10/site-packages/ipykernel/iostream.py:575 th=MainThread
	 { OutStream.write /home/jjkim/.local/lib/python3.10/site-packages/ipykernel/iostream.py:535 th=MainThread
		 { OutStream._is_master_process /home/jjkim/.local/lib/python3.10/site-packages/ipykernel/iostream.py:444 th=Mai

### with decorator

In [ ]:
import sys

import threading
import time
from scope_fn import scope


class Middle:
    @scope.fn
    def __init__(self, b) -> None:
        print('b', b, ' in Base')

class A(Middle):
    @scope.fn
    def __init__(self, a=1, b=0) -> None:
        super().__init__(b)

    @scope.fn
    def memberfunc(self, a,b):
        print('memberfunc', a,b)


@scope.fn
def scope_test():

    print('-------------- max_count=2 test ---------------')
    @scope.fn
    def max_count_test(a,b,c):
        print(f'It will shown only max_count(2) times', a,b,c)

        obj = A(100,101)
        obj.memberfunc(4,5)
        return a+b+c

    for _ in range(5):
        max_count_test(1,2,3)

    print('-------------- suppress test ---------------')

    @scope.fn
    def in_suppress_1():
        print('Not seen: in_suppress_1')

    @scope.fn
    def suppress_following():
        print('enter: suppress_following()')
        in_suppress_1()
        print('exit: suppress_following: after in_suppress_1')
        return
    
    suppress_following()
    return

@scope.fn
def print_numbers():

    for i in range(1, 3):
        print(f"number: {i}")
        scope_test()
        time.sleep(1)

@scope.fn
def print_letters():

    for letter in 'abcde':
        print(f"Letter: {letter}")
        scope_test()
        time.sleep(1)


if __name__ == "__ipynb__":

    threads = []
    for i, fn in enumerate([print_numbers, print_letters]):
        t = threading.Thread(target=fn, name=f't{i}')
        threads.append(t)
        t.start()

    for t in threads: t.join()



## deco

In [ ]:
import time

def decorator_function(the_func, called=0):
    def wrapper(*args, **kwargs):
        nonlocal called
        tick = time.time()
        result = the_func(*args, **kwargs)
        print(f"{the_func.__name__}: {called=}")
        called += 1
        return result
    return wrapper

def real_func():
    print('real_func: body')




real_func_var = decorator_function(real_func)  # 1

real_func_var()
real_func_var()

@decorator_function
def real_func2():
    print('real_func2: body')

real_func2()

In [ ]:

class decorator_class:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        tick = time.time()
        result = self.func(*args, **kwargs)
        print(f"{self.func.__name__}: exec={time.time() - tick}sec")
        return result


def real_func():
    print('real_func: called')



real_func = decorator_class(real_func)  # 1

real_func()


In [ ]:
from dataclasses import dataclass
import enum

MAX_FUNC_LOGGING = 999999


@enum.unique
class Suppress(enum.Enum):
    SelfOnly = enum.auto()
    Following = enum.auto()
    SelfAndFollowing = enum.auto()


class Trace:
    _workdir = None

    Suppress = Suppress
    @dataclass
    class FuncContext:
        # id:str
        show_args:bool = False
        logging_cnt:int=MAX_FUNC_LOGGING
        suppress:Suppress = None


print(Trace.Suppress.Following)
print(Suppress)


### similarity score:
- to remove similar image from dataset.

In [46]:
import re
str1 = '''
            { Struct.__init__() utils abc/masking1.py:40, from Masking.__init__ (utils/masking.py:57) - MainThread
            { Masking.__init__() utils/masking2.py:52, from BaseModel.__init__ (micalib/base_model.py:43) - MainThread
'''

matches = re.findall(r'\(\)\s+((?!\.py).+?\.py):\d+', str1)# , flags=re.MULTILINE)
pyfiles = matches
print(type(pyfiles), pyfiles)


str2 = '''
IBasicBlock.forward() models/arcface.py:61, from
{ Masking.__init__() utils/masking2.py:52, from BaseModel.__init__ (micalib/base_model.py:43) - MainThread
                              IBasicBlock.__init__() models/arcface.py:42, fr
                              Masking.get_mask_eyes() utils abc/masking.py:87, f
                             # to_np() models/flame.py:26, f                    -> THIS LINE WILL BE IGNORED
                               to_tensor() models/flame.py:21, fr
                                { <lambda>() utils/masking.py:75,
'''

matches = re.findall(r'([\w<](?!\(\)).+?)\(\)\s+([^:]+):\d+', str2)# , flags=re.MULTILINE)
pyfiles = matches
for (fn,file) in matches:
    print(fn, file)

<class 'list'> ['utils abc/masking1.py', 'utils/masking2.py']
IBasicBlock.forward models/arcface.py
Masking.__init__ utils/masking2.py
IBasicBlock.__init__ models/arcface.py
Masking.get_mask_eyes utils abc/masking.py
to_np models/flame.py
to_tensor models/flame.py
<lambda> utils/masking.py
